### Build Docker Image that contains Sound Classification model and Flask web application
Make sure you are able to run Docker without sudo.  


In [1]:
import os
from os import path
import json

In [19]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

mkdir: cannot create directory ‘flaskwebapp’: File exists
mkdir: cannot create directory ‘flaskwebapp/nginx’: File exists
mkdir: cannot create directory ‘flaskwebapp/etc’: File exists


###Get the sound Classification model

This is not part of this jupyter notebook

In [ ]:
Below is the driver for our model. The methods in this module will be called by our webapp.

In [5]:
%%writefile flaskwebapp/driver.py
import numpy as np
import logging, sys, json
import timeit as t
import base64
from PIL import Image, ImageOps
from io import BytesIO
import urllib.request
from keras.models import load_model
import librosa


logger = logging.getLogger("cntk_svc_logger")
ch = logging.StreamHandler(sys.stdout)
logger.addHandler(ch)

trainedModel = None
mem_after_init = None
labelLookup = None
topResult = 3

sound_names = ["air conditioner","car horn","children playing","dog bark","drilling","engine idling",
               "gun shot","jackhammer","siren","street music"]

def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features_array(filename, bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    sound_clip,s = librosa.load(filename)        
    for (start,end) in windows(sound_clip,window_size):
        start = int(start)
        end = int(end)
        if(len(sound_clip[start:end]) == window_size):
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            logspec = librosa.logamplitude(melspec)
            logspec = logspec.T.flatten()[:, np.newaxis].T
            log_specgrams.append(logspec)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features)


def init():
    """ Initialise ResNet 152 model
    """
    global trainedModel, labelLookup, mem_after_init

    start = t.default_timer()
    print("model initializing")
    
    model_filepath = "/code/salamon-cnn-model-xiaoyzhu.h5"
    trainedModel = load_model(model_filepath)
    print("model loaded. model is", trainedModel)
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)


def run(inputString):
    """ Classify the input using the loaded model
    """
    start = t.default_timer()

    print("downloading with urllib, url is ", inputString)
    urllib.request.urlretrieve(inputString, "./test.wav")
    
    # load audio file and extract features
    predict_file = "./test.wav"
    predict_x = extract_features_array(predict_file)
    
    # generate prediction, passing in just a single row of features
    predictions = trainedModel.predict(predict_x)
    
    if len(predictions) == 0: 
        print("No prediction")


    ind = np.argpartition(predictions[0], -2)[-2:]
    ind[np.argsort(predictions[0][ind])]
    ind = ind[::-1]
    
    print("Top guess: ", sound_names[ind[0]], " (",round(predictions[0,ind[0]],3),")")
    print("2nd guess: ", sound_names[ind[1]], " (",round(predictions[0,ind[1]],3),")")
    
    end = t.default_timer()
    
    logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
    return (sound_names[ind[0]], sound_names[ind[1]], 'Computed in {0} ms'.format(round((end-start)*1000, 2)))


Overwriting flaskwebapp/driver.py


Below is the module for the Flask web application.

In [6]:
%%writefile flaskwebapp/app.py
from flask import Flask, request
import tensorflow as tf
from driver import *
import time

app = Flask(__name__)


@app.route('/score', methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        print("request received")
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    payload = request.get_json()
    input = json.loads(payload)['input']
    start = time.time()
    print("input is", input)
    response = run(input)
    end = time.time() - start
    dict = {}
    dict['result'] = response
    return json.dumps(dict)


@app.route("/")
def healthy():
    return "Healthy"


# CNTK Version
@app.route('/version', methods = ['GET'])
def version_request():
    return tf.__version__


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server

Overwriting flaskwebapp/app.py


In [7]:
%%writefile flaskwebapp/wsgi.py
import sys
sys.path.append('/code/') # FIXME: This is horrible
from app import app as application
from driver import *

def create():
    print("Initialising")
    init()
    application.run(host='127.0.0.1', port=5000)

Overwriting flaskwebapp/wsgi.py


In [8]:
%%writefile flaskwebapp/requirements.txt
numpy
pillow
click==6.7
configparser==3.5.0
Flask==0.11.1
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3
tensorflow
keras
h5py

Overwriting flaskwebapp/requirements.txt


The configuration for the Nginx. Note that it creates a proxy between ports **88** and **5000**.

In [9]:
%%writefile flaskwebapp/nginx/app
server {
    listen 88;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Overwriting flaskwebapp/nginx/app


In [10]:
image_name = "xiaoyzhu/soundclassification"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [11]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Overwriting flaskwebapp/gunicorn_logging.conf


In [12]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()


Overwriting flaskwebapp/kill_supervisor.py


In [13]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Overwriting flaskwebapp/etc/supervisord.conf


We create a custom image based on Ubuntu 16.04 and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [14]:
%%writefile flaskwebapp/dockerfile

FROM ubuntu:16.04
LABEL maintainer = "Mathew Salvaris <mathew.salvaris@microsoft.com>"
LABEL modifiedBy = "xiaoyzhu"

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update
RUN apt-get install -y --no-install-recommends \
        openmpi-bin \
        python3 \
        apt-utils \
        python3-dev \
        python3-setuptools \
        python3-pip \
        supervisor \
        nginx \
        build-essential \
        autoconf \
        libtool \
        libav-tools \
        pkg-config
        

RUN rm /etc/nginx/sites-enabled/default 
RUN cp /code/nginx/app /etc/nginx/sites-available/ 
RUN ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/

RUN pip3 install --upgrade pip
RUN pip3 install wheel cython numpy
RUN pip3 install -r /code/requirements.txt
RUN pip3 install librosa

EXPOSE 88

RUN alias python=python3 
CMD ["supervisord", "-c", "/etc/supervisord.conf"]

Overwriting flaskwebapp/dockerfile


In [15]:
# image_name = "xiaoyzhu/cntkresnet"
# application_path = 'flaskwebapp'
# docker_file_location = path.join(application_path, 'dockerfile')
#!docker build -t $image_name -f $docker_file_location $application_path --no-cache

In [16]:
!docker build -t $image_name  $application_path --no-cache

Sending build context to Docker daemon  244.2MB
Step 1/19 : FROM ubuntu:16.04
 ---> ccc7a11d65b1
Step 2/19 : LABEL maintainer = "Mathew Salvaris <mathew.salvaris@microsoft.com>"
 ---> Running in a0bdcb0ebbe5
 ---> f5dfb6b5d227
Removing intermediate container a0bdcb0ebbe5
Step 3/19 : LABEL modifiedBy = "xiaoyzhu"
 ---> Running in cf07e423edcd
 ---> 42c87b9634f3
Removing intermediate container cf07e423edcd
Step 4/19 : RUN mkdir /code
 ---> Running in 3a5be72a368e
 ---> c16a37b87359
Removing intermediate container 3a5be72a368e
Step 5/19 : WORKDIR /code
 ---> 51b9bc92f3cf
Removing intermediate container f48d7bcc951d
Step 6/19 : ADD . /code/
 ---> 554eb0a7f083
Removing intermediate container 86f7bd3879fe
Step 7/19 : ADD etc /etc
 ---> 01e6459c7221
Removing intermediate container ad0d0ca2fb24
Step 8/19 : RUN apt-get update
 ---> Running in 80d3e33d8192
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
G

In [18]:
print('Docker image name {}'.format(image_name)) 

Docker image name xiaoyzhu/soundclassification


### Test locally
Go to the [Test Locally notebook](TestLocally.ipynb) to test your Docker image